# Offline Engine API

SGLang provides a direct inference engine without the need for an HTTP server, especially for use cases where additional HTTP server adds unnecessary complexity or overhead. Here are two general use cases:

- Offline Batch Inference
- Custom Server on Top of the Engine

This document focuses on the offline batch inference, demonstrating four different inference modes:

- Non-streaming synchronous generation
- Streaming synchronous generation
- Non-streaming asynchronous generation
- Streaming asynchronous generation

Additionally, you can easily build a custom server on top of the SGLang offline engine. A detailed example working in a python script can be found in [custom_server](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/custom_server.py).



## Nest Asyncio
Note that if you want to use **Offline Engine** in ipython or some other nested loop code, you need to add the following code:
```python
import nest_asyncio

nest_asyncio.apply()

```

## Advanced Usage

The engine supports [vlm inference](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/offline_batch_inference_vlm.py) as well as [extracting hidden states](https://github.com/sgl-project/sglang/blob/main/examples/runtime/hidden_states). 

Please see [the examples](https://github.com/sgl-project/sglang/tree/main/examples/runtime/engine) for further use cases.

## Offline Batch Inference

SGLang offline engine supports batch inference with efficient scheduling.

In [1]:
# launch the offline engine
import asyncio

import sglang as sgl
import sglang.test.doc_patch
from sglang.utils import async_stream_and_merge, stream_and_merge

llm = sgl.Engine(model_path="qwen/qwen2.5-0.5b-instruct")

[2026-02-16 17:04:11] INFO utils.py:148: Note: detected 128 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.


[2026-02-16 17:04:11] INFO utils.py:151: Note: NumExpr detected 128 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 16.


[2026-02-16 17:04:11] INFO utils.py:164: NumExpr defaulting to 16 threads.


<frozen importlib._bootstrap_external>:1184: FutureWarning: The cuda.cudart module is deprecated and will be removed in a future release, please switch to use the cuda.bindings.runtime module instead.
<frozen importlib._bootstrap_external>:1184: FutureWarning: The cuda.nvrtc module is deprecated and will be removed in a future release, please switch to use the cuda.bindings.nvrtc module instead.


[2026-02-16 17:04:13] INFO server_args.py:1830: Attention backend not specified. Use fa3 backend by default.


...

Capturing batches (bs=120 avail_mem=59.59 GB):   5%|▌         | 1/20 [00:00<00:03,  5.52it/s]

Capturing batches (bs=72 avail_mem=59.59 GB):  35%|███▌      | 7/20 [00:00<00:00, 20.26it/s]

Capturing batches (bs=24 avail_mem=59.58 GB):  65%|██████▌   | 13/20 [00:00<00:00, 23.79it/s]

Capturing batches (bs=1 avail_mem=59.57 GB):  80%|████████  | 16/20 [00:00<00:00, 22.95it/s]

Capturing batches (bs=1 avail_mem=59.57 GB): 100%|██████████| 20/20 [00:00<00:00, 22.42it/s]


### Non-streaming Synchronous Generation

In [2]:
prompts = [
    "Hello, my name is",
    "The president of the United States is",
    "The capital of France is",
    "The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Hello, my name is
Generated text:  Marco. I'm a 15-year-old boy from Italy. I don't have a phone or a computer. I usually use my phone to write email and send messages to my friends. But I don't like the way my phone is used by many people. Now, I am writing a story. I like to write letters, but I don't like the way I type letters. Sometimes I want to write letters but I don't have a pen to write them. I like to write letters to my family or my friends. And I think letters are very important. They're not just for talking about what I'm going to
Prompt: The president of the United States is
Generated text:  3 feet tall and has a weight of 200 pounds. If the president decides to lose 10% of his body weight, what will be the president's new height? To determine the president's new height after losing 10% of his body weight, we need to follow these steps:

1. Calculate 10% of the president's current weight.
2. Subtract this value from the president's current weight.

First, let's f

### Streaming Synchronous Generation

In [3]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {
    "temperature": 0.2,
    "top_p": 0.9,
}

print("\n=== Testing synchronous streaming generation with overlap removal ===\n")

for prompt in prompts:
    print(f"Prompt: {prompt}")
    merged_output = stream_and_merge(llm, prompt, sampling_params)
    print("Generated text:", merged_output)
    print()


=== Testing synchronous streaming generation with overlap removal ===

Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is


Generated text:  [Name], and I'm a [Job Title] at [Company Name]. I'm excited to meet you and learn more about you. What can you tell me about yourself? I'm a [insert a short description of your job or experience here]. I'm always looking for new challenges and opportunities to grow and learn. What's your favorite hobby or activity? I enjoy [insert a short description of your favorite hobby or activity here]. I'm always looking for ways to stay active and healthy, so I'm always on the lookout for new fitness programs or activities. What's your favorite book or movie? I love [insert a short

Prompt: Provide a concise factual statement about France’s capital city. The capital of France is


Generated text:  Paris, the city that serves as the political, cultural, and economic center of the country. It is known for its iconic landmarks such as the Eiffel Tower, Notre-Dame Cathedral, and the Louvre Museum, as well as its rich history and diverse culture. Paris is also a major tourist destination, with millions of visitors each year. The city is home to many famous French artists, writers, and musicians, and is known for its cuisine, including its famous Parisian dishes such as croissants and baguettes. Overall, Paris is a vibrant and dynamic city that continues to be a major center of French culture and

Prompt: Explain possible future trends in artificial intelligence. The future of AI is


Generated text:  likely to be characterized by rapid advancements in several key areas, including:

1. Increased integration with human intelligence: AI is likely to become more integrated with human intelligence, allowing machines to learn and adapt to human behavior and preferences. This could lead to more personalized and adaptive AI systems that can better understand and respond to human needs.

2. Enhanced machine learning capabilities: AI is likely to become even more powerful and capable, with the ability to learn from vast amounts of data and make more accurate predictions and decisions. This could lead to more efficient and effective use of resources, as well as more accurate predictions of human behavior.

3. Greater reliance on



### Non-streaming Asynchronous Generation

In [4]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous batch generation ===")


async def main():
    outputs = await llm.async_generate(prompts, sampling_params)

    for prompt, output in zip(prompts, outputs):
        print(f"\nPrompt: {prompt}")
        print(f"Generated text: {output['text']}")


asyncio.run(main())


=== Testing asynchronous batch generation ===



Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is
Generated text:  [Name] and I am a [Title]. I am passionate about [What makes you passionate about your job/field/interest/whatever]. I am currently [Your current occupation, position, or role]. I am [Your age, gender, etc.]. I like to [Describe your hobbies, interests, or talents]. I love [Explain why this makes me unique and special]. I am always ready to learn new things and continue my education. I believe in [What is your career goal or dream for your life]. I have always been [Tell me about your life experiences, if anything]. I am patient, hardworking

Prompt: Provide a concise factual statement about France’s capital city. The capital of France is
Generated text:  Paris, known as the "City of Love" and the "Paris of the Arts". 

1. **Paris, known as the "City of Love" and the "Paris of the Arts", is the capital of France.**
2. It's the largest and most populous city in France.
3. The 

### Streaming Asynchronous Generation

In [5]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous streaming generation (no repeats) ===")


async def main():
    for prompt in prompts:
        print(f"\nPrompt: {prompt}")
        print("Generated text: ", end="", flush=True)

        # Replace direct calls to async_generate with our custom overlap-aware version
        async for cleaned_chunk in async_stream_and_merge(llm, prompt, sampling_params):
            print(cleaned_chunk, end="", flush=True)

        print()  # New line after each prompt


asyncio.run(main())


=== Testing asynchronous streaming generation (no repeats) ===

Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is
Generated text: 

 [

Your

 Name

],

...

 to

 more

 personalized

 and

 effective

In [6]:
llm.shutdown()